# RobotInterfaceの使い方

In [1]:
exec(open('/choreonoid_ws/install/share/irsl_choreonoid/sample/irsl_import.py').read())

## インターフェースの初期化

In [2]:
ri = RobotInterface('/userdir/robotinterface.yaml', connection_wait=3.0)

[INFO] [1693892666.537528, 0.000000]: loading model from //userdir/samplerobot.body
joint: [{'name': 'default', 'topic': '/samplerobot/joint_controller/command', 'type': 'command', 'joint_names': ['yaw_joint', 'pitch_joint']}]
devices: [{'topic': '/samplerobot/joint_states', 'class': 'JointState', 'name': 'joint_state'}, {'topic': '/samplerobot/joint_controller/state', 'class': 'JointTrajectoryState', 'name': 'joint_trajectory_state'}]

## インターフェースのロボットモデルを使う
- クラスを登録しておくと良い
- クラス化が無ければ、都度register(end-effector, pose)する

関節全体を'arm'として登録している。'joint2'は終端のジョイントになります。

robotオブジェクトは「ロボットモデルの使い方」で説明したものと同じです。

In [4]:
robot = ri.getRobotModel()

In [5]:
robot.jointNames

['yaw_joint', 'pitch_joint']

下の'pitch_joint'はjointNamesで返ってきた名前に変更する

In [6]:
robot.registerEndEffector('arm', 'pitch_joint', joint_list=robot.jointNames)

## 関節の情報

### jointGroupについて

グループの確認

In [7]:
ri.jointGroupList

グループの名前

In [8]:
ri.jointGroupNames

['default']

デフォルトグループの関節

In [9]:
ri.getJointGroup('default').jointNames

['yaw_joint', 'pitch_joint']

In [10]:
ri.getJointGroup('default') is ri.jointGroupList[0]

True

### 関節に関するセンシング情報

センシングした関節角度

In [11]:
ri.actualAngleVector

array([0., 0.])

参照値（コマンドで送っている値、現状ではassemblerの実機では取れない）

In [12]:
ri.referenceAngleVector

array([0., 0.])

速度

In [13]:
ri.velocityVector

array([0., 0.])

エフォート（トルク、力）

In [14]:
ri.effortVector

array([0., 0.])

## ロボットへ関節角度を送る

### モデルの姿勢を実機へ送る

モデルへ角度セット、名前指定 (表示が変わる)

In [17]:
robot.setAngleMap({'yaw_joint': -0.4})

True

角度を送る

```ri.sendAngleVector(<angle-vector>, <動作時間[秒]>)```

In [18]:
ri.sendAngleVector(robot.angleVector(), 4.0)

動作終了を待つ

```ri.waitUntilFinish(<タイムアウト[秒]>)```

In [19]:
ri.waitUntilFinish(5.0)

モデルへ角度セット、ベクトル指定(表示が変わる)

In [21]:
robot.angleVector(fv(0.8, 0.8))

角度を送る

In [22]:
ri.sendAngleVector(robot.angleVector(), 10.0)

動作終了を待つ

In [23]:
ri.waitUntilFinish(11.0)

### IKを使って姿勢を変える

In [28]:
cds = robot.arm.endEffector

In [29]:
cds.translate(fv(-0.04, 0, 0))

<coordinates[0x56304409db70] 0.0376469 0.0810854 0.0555058 / -3.29688e-07 -0.707106 -0.492646 0.507248 >

In [30]:
robot.arm.inverseKinematics(cds, constraint='x')

(True, 5)

In [31]:
ri.sendAngleVector(robot.angleVector(), 2.0);

In [32]:
ri.waitUntilFinish(3.0)

## 台車を動かす

台車を移動させる

```ri.move_velocity(<x軸方向速度>, <y軸方向速度>, <z軸回転角速度>)```

In [ ]:
ri.move_velocity(0.1, 0, 0)

waitする。現状はスリープしている。

In [ ]:
ri.waitFinishMoving(2.0)

動作停止

In [ ]:
ri.stop()

## センサーデータの使い方

### センサデバイスの名前

In [33]:
ri.deviceNames

['joint_state', 'joint_trajectory_state']

### データを受け取る
- バッファにデータがあればデータを返す
- バッファにデータが無ければNoneが返る
- clearするとバッファからデータを除く

（前提: データを受け取るとバッファに入る）

In [34]:
data = ri.data('joint_state', clear=False)
data

header: 
  seq: 17824
  stamp: 
    secs: 356
    nsecs: 762999999
  frame_id: ''
name: 
  - pitch_joint
  - yaw_joint
position: [0.7999999999999998, -0.3114480729526705]
velocity: [0.0, 0.0]
effort: [0.0, 0.0]

### データが来るまで待つ
- バッファにデータがあればデータを返す
- バッファにデータが無ければtimeoutするまで待つ
  - データが来れば返す、来なければNoneが返る
- clearするとバッファからデータを除く

In [35]:
data = ri.waitData('joint_state', timeout=1.0, clear=False)
data

header: 
  seq: 18335
  stamp: 
    secs: 366
    nsecs: 983000000
  frame_id: ''
name: 
  - pitch_joint
  - yaw_joint
position: [0.7999999999999998, -0.3114480729526705]
velocity: [0.0, 0.0]
effort: [0.0, 0.0]

### 次のデータが来るまで待つ
- バッファのデータと無関係に、データが来るのをtimeoutするまで待つ
  - データが来れば返す、来なければNoneが返る
- clearするとバッファからデータを除く  


In [36]:
data = ri.waitNextData('joint_state', timeout=1.0, clear=False)
data

header: 
  seq: 19076
  stamp: 
    secs: 381
    nsecs: 802999999
  frame_id: ''
name: 
  - pitch_joint
  - yaw_joint
position: [0.7999999999999998, -0.3114480729526705]
velocity: [0.0, 0.0]
effort: [0.0, 0.0]

## ループの作り方

In [ ]:
import rospy
while rospy.ok():
    ## <programs>
    rospy.sleep(0.1)
